<a href="https://colab.research.google.com/github/ChristianParsons98/PG_Lab_Code/blob/main/Chem_Mass_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
from sympy import Matrix
from sympy import symbols, Eq, solve
import re

In [18]:
############################################################
# Created: February 11 2025                                #
# by: Christian Parsons                                    #
# Updated: February 11 2025                                #
# Authors: Christian Parsons                               #
#                                                          #
#                                                          #
# This notebook is used to calculate the mass              #
# of input chemicals for a given output                    #
#                                                          #
############################################################

In [19]:
#Creating a dictionary of the chemical element abreviations as the key and the atomic mass as the value.
atomic_masses = {
    "H": 1.008, "He": 4.0026, "Li": 6.94, "Be": 9.0122, "B": 10.81, "C": 12.011, "N": 14.007, "O": 15.999,
    "F": 18.998, "Ne": 20.180, "Na": 22.990, "Mg": 24.305, "Al": 26.982, "Si": 28.085, "P": 30.974, "S": 32.06,
    "Cl": 35.45, "Ar": 39.948, "K": 39.098, "Ca": 40.078, "Sc": 44.956, "Ti": 47.867, "V": 50.941, "Cr": 51.996,
    "Mn": 54.938, "Fe": 55.845, "Co": 58.933, "Ni": 58.693, "Cu": 63.546, "Zn": 65.38, "Ga": 69.723, "Ge": 72.630,
    "As": 74.922, "Se": 78.971, "Br": 79.904, "Kr": 83.798, "Rb": 85.468, "Sr": 87.62, "Y": 88.906, "Zr": 91.224,
    "Nb": 92.906, "Mo": 95.95, "Tc": 98.0, "Ru": 101.07, "Rh": 102.91, "Pd": 106.42, "Ag": 107.87, "Cd": 112.41,
    "In": 114.82, "Sn": 118.71, "Sb": 121.76, "Te": 127.60, "I": 126.90, "Xe": 131.29, "Cs": 132.91, "Ba": 137.33,
    "La": 138.91, "Ce": 140.12, "Pr": 140.91, "Nd": 144.24, "Pm": 145.0, "Sm": 150.36, "Eu": 151.96, "Gd": 157.25,
    "Tb": 158.93, "Dy": 162.50, "Ho": 164.93, "Er": 167.26, "Tm": 168.93, "Yb": 173.05, "Lu": 174.97, "Hf": 178.49,
    "Ta": 180.95, "W": 183.84, "Re": 186.21, "Os": 190.23, "Ir": 192.22, "Pt": 195.08, "Au": 196.97, "Hg": 200.59,
    "Tl": 204.38, "Pb": 207.2, "Bi": 208.98, "Th": 232.04, "Pa": 231.04, "U": 238.03, "Np": 237.0, "Pu": 244.0,
    "Am": 243.0, "Cm": 247.0, "Bk": 247.0, "Cf": 251.0, "Es": 252.0, "Fm": 257.0, "Md": 258.0, "No": 259.0,
    "Lr": 266.0, "Rf": 267.0, "Db": 270.0, "Sg": 271.0, "Bh": 270.0, "Hs": 277.0, "Mt": 278.0, "Ds": 281.0,
    "Rg": 282.0, "Cn": 285.0, "Nh": 286.0, "Fl": 289.0, "Mc": 290.0, "Lv": 293.0, "Ts": 294.0, "Og": 294.0
}

In [20]:
#Parses a chemical formula into a dictionary of element counts.
#Example: parse_formula("Na3PS4") -> {"Na": 3, "P": 1, "S": 4}

def parse_formula(formula):
    element_pattern = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    return {el: int(count) if count else 1 for el, count in element_pattern}

In [21]:
print(parse_formula("Na3PS4"))

{'Na': 3, 'P': 1, 'S': 4}


In [22]:
#This function finds the correct ratio of reactants to produce one mole of the given product.
#Give the product as a list of dictionaries of the form [{"H": 2, "O": 1}]
#and the reactants as a list of dictionaries the form [{"H": 2, "O": 1},{"H": 2, "O": 1}].


def find_stoichiometric_ratios(reactant_compositions, product_compositions):
    """
    Finds the correct stoichiometric ratios for a balanced chemical reaction using matrix algebra.

    Parameters:
    reactant_compositions (list): List of dictionaries representing reactants' compositions.
    product_compositions (list) : List of dictionaries representing products' compositions.

    Returns:
    tuple: Two lists containing the stoichiometric coefficients for reactants and products.
    """
    all_species = reactant_compositions + product_compositions
    num_reactants = len(reactant_compositions)
    num_products = len(product_compositions)

    # Extract all unique elements involved in the reaction
    elements = set()
    for species in all_species:
        elements.update(species.keys())

    elements = sorted(elements)  # Keep element order consistent
    num_elements = len(elements)

    # Construct the stoichiometric matrix
    matrix = np.zeros((num_elements, num_reactants + num_products))

    for i, element in enumerate(elements):
        for j, species in enumerate(reactant_compositions):
            matrix[i, j] = species.get(element, 0)  # Reactants are positive
        for j, species in enumerate(product_compositions):
            matrix[i, num_reactants + j] = -species.get(element, 0)  # Products are negative

    # Convert to sympy matrix to find null space (solution space)
    M = Matrix(matrix)
    null_space = M.nullspace()

    if not null_space:
        return "No valid solution found."

    # Extract the smallest integer solution
    coeffs = np.array(null_space[0]).astype(float).flatten()
    coeffs = np.abs(coeffs)  # Ensure positive coefficients
    coeffs /= np.min(coeffs[coeffs > 0])  # Normalize to the smallest integer coefficient

    # Split coefficients into reactants and products
    reactant_values = coeffs[:num_reactants]
    product_values = coeffs[num_reactants:]

    return reactant_values.tolist(), product_values.tolist()

In [23]:
# Example 1:
reactants_hashed = [{"C": 1, "O": 2}, {"H": 2, "O": 1}]  # CO2, H2O
products_hashed = [{"C": 1, "H": 4}, {"O": 2}]  # CH4, O2
ratios = find_stoichiometric_ratios(reactants_hashed, products_hashed)
print(ratios)  # Expected Output: ([1.0, 2.0], [1.0, 2.0]) for CO2 + 2H2O -> CH4 + 2O2

([1.0, 2.0], [1.0, 2.0])


In [24]:
# Example #2:
product_hashed = [{"Na": 3, "P": 1,"S":4}]  # Na3PS4
reactants_hashed = [{"P": 2,"S":5}, {"Na": 2,"S":1}]  # P2S5, Na2S1
ratios = find_stoichiometric_ratios(product_hashed, reactants_hashed)
print(ratios)  # Output: ([2.0], [1.0, 3.0])

([2.0], [1.0, 3.0])


In [25]:
def compute_mass_ratios(reactants_hashed, products_hashed, target_product_mass, product_index=0):

    def get_molar_mass(compound):
        #"""Computes the molar mass of a chemical compound."""
        return sum(atomic_masses[element] * count for element, count in compound.items())

    # Get balanced reaction coefficients
    reactant_ratios, product_ratios = find_stoichiometric_ratios(reactants_hashed, products_hashed)

    if isinstance(reactant_ratios, str):  # If balancing failed
        return "No valid reaction found."

    # Compute molar masses
    product_molar_masses = [get_molar_mass(comp) for comp in products_hashed]
    reactant_molar_masses = [get_molar_mass(comp) for comp in reactants_hashed]

    # Select the specified product as reference
    reference_molar_mass = product_molar_masses[product_index]  # Molar mass of the reference product
    reference_ratio = product_ratios[product_index]  # Stoichiometric coefficient of reference product

    # Mass of 1 mole of the selected product
    mass_per_mole_product = reference_ratio * reference_molar_mass

    # Scale factor for the target product mass
    scale_factor = target_product_mass / mass_per_mole_product

    # Compute mass of reactants required
    reactant_masses = {
        f"Reactant {i+1}": reactant_ratios[i] * reactant_molar_masses[i] * scale_factor
        for i in range(len(reactants_hashed))
    }

    # Compute mass of other products produced
    product_masses = {
        f"Product {i+1}": product_ratios[i] * product_molar_masses[i] * scale_factor
        for i in range(len(products_hashed))
    }

    return reactant_masses, product_masses

In [26]:
# Example usage:
reactants_hashed = [{"C": 1, "O": 2}, {"H": 2, "O": 1}]  # CO2, H2O
products_hashed = [{"C": 1, "H": 4}, {"O": 2}]  # CH4, O2
target_mass = 10  # 10g of CH4

reactant_masses, product_masses = compute_mass_ratios(reactants_hashed, products_hashed, target_mass)
print("Mass of reactants needed:", reactant_masses)
print("Mass of products produced:", product_masses)

Mass of reactants needed: {'Reactant 1': 27.43190176400923, 'Reactant 2': 22.458393068628066}
Mass of products produced: {'Product 1': 10.0, 'Product 2': 39.890294832637295}


In [27]:
# Test:
reactants_hashed = [{"Ca": 1}, {"O": 2}]  # Ca, O2
products_hashed = [{"Ca": 1, "O": 1}]  # CaO2
ratios = find_stoichiometric_ratios(reactants_hashed, products_hashed)
print(ratios)  # Expected Output: ([2.0, 1.0], [2.0]) for 2Ca + O2 -> 2CaO

target_mass = 51  # 10g of CH4

reactant_masses, product_masses = compute_mass_ratios(reactants_hashed, products_hashed, target_mass)
print("Mass of reactants needed:", reactant_masses)
print("Mass of products produced:", product_masses)

([2.0, 1.0], [2.0])
Mass of reactants needed: {'Reactant 1': 36.44948909535103, 'Reactant 2': 14.550510904648965}
Mass of products produced: {'Product 1': 51.0}


In [32]:
def compute_mass_ratios(reactants_hashed, products_hashed, reactant_ratios, product_ratios, target_product_mass, product_index=0):

    def get_molar_mass(compound):
        """Computes the molar mass of a chemical compound."""
        return sum(atomic_masses[element] * count for element, count in compound.items())

    # Compute molar masses of all reactants and products
    product_molar_masses = [get_molar_mass(comp) for comp in products_hashed]
    reactant_molar_masses = [get_molar_mass(comp) for comp in reactants_hashed]

    # Select the specified product as reference
    reference_molar_mass = product_molar_masses[product_index]  # Molar mass of the reference product
    reference_ratio = product_ratios[product_index]  # Stoichiometric coefficient of reference product

    # Mass of 1 mole of the selected product
    mass_per_mole_product = reference_ratio * reference_molar_mass

    # Scale factor for the target product mass
    scale_factor = target_product_mass / mass_per_mole_product

    # Compute mass of reactants required
    reactant_masses = {
        f"Reactant {i+1}": reactant_ratios[i] * reactant_molar_masses[i] * scale_factor
        for i in range(len(reactants_hashed))
    }

    # Compute mass of other products produced
    product_masses = {
        f"Product {i+1}": product_ratios[i] * product_molar_masses[i] * scale_factor
        for i in range(len(products_hashed))
    }

    return reactant_masses, product_masses


In [33]:
reactants_hashed = [{"Ca": 1}, {"O": 2}]  # Ca, O2
products_hashed = [{"Ca": 1, "O": 1}]  # CaO2

# Get the balanced reaction coefficients
reactant_ratios, product_ratios = find_stoichiometric_ratios(reactants_hashed, products_hashed)

# Specify the desired product mass
target_mass = 53  # 10g of CH4

# Compute mass ratios
reactant_masses, product_masses = compute_mass_ratios(
    reactants_hashed, products_hashed, reactant_ratios, product_ratios, target_mass
)

#Answer should be {'Reactant 1': 37.878880824580484, 'Reactant 2': 15.121119175419512}
print("Mass of reactants needed:", reactant_masses)
print("Mass of products produced:", product_masses)

Mass of reactants needed: {'Reactant 1': 37.878880824580484, 'Reactant 2': 15.121119175419512}
Mass of products produced: {'Product 1': 53.0}
